# CellStrata: Loading and Exploring Downloaded Census Data

This notebook demonstrates how to:
1. Load downloaded .h5ad files (single or batched)
2. Explore the data structure
3. Merge batches if needed
4. Basic QC metrics

In [ ]:
# Import libraries
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Set scanpy settings
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, facecolor='white')

print(f"Scanpy version: {sc.__version__}")

## Option 1: Load a Single .h5ad File

In [ ]:
# Path to your downloaded file
# Change this to match your file location

# Single file
h5ad_file = "data/raw/dataset_1b350d0a.h5ad"

# Or a batch file
# h5ad_file = "data/raw/batches/batch_0000.h5ad"

# Or a chunk file
# h5ad_file = "data/raw/chunks/chunk_mast_cell.h5ad"

In [ ]:
# Load the file
adata = sc.read_h5ad(h5ad_file)
print(f"Loaded: {adata.n_obs:,} cells x {adata.n_vars:,} genes")

In [ ]:
# View the AnnData structure
adata

## Option 2: Load and Merge Multiple Batches

In [ ]:
# Find all batch files
batch_dir = Path("data/raw/batches")
batch_files = sorted(batch_dir.glob("batch_*.h5ad"))

print(f"Found {len(batch_files)} batch files:")
for f in batch_files[:5]:
    print(f"  {f.name}")
if len(batch_files) > 5:
    print(f"  ... and {len(batch_files) - 5} more")

In [ ]:
# Load and merge batches (memory-efficient)
# Only run this if you have enough RAM!

if len(batch_files) > 0:
    print("Loading batches...")
    
    # Load first batch
    adata = sc.read_h5ad(batch_files[0])
    print(f"  Loaded {batch_files[0].name}: {adata.n_obs:,} cells")
    
    # Concatenate remaining batches
    for batch_file in batch_files[1:]:
        batch = sc.read_h5ad(batch_file)
        print(f"  Loaded {batch_file.name}: {batch.n_obs:,} cells")
        adata = sc.concat([adata, batch], join='outer')
        del batch  # Free memory
    
    print(f"\nMerged: {adata.n_obs:,} cells x {adata.n_vars:,} genes")
else:
    print("No batch files found. Using single file instead.")

## Explore the Data Structure

In [ ]:
# View cell metadata columns
print("Cell metadata columns (adata.obs):")
print(adata.obs.columns.tolist())

In [ ]:
# View first few rows of cell metadata
adata.obs.head()

In [ ]:
# View gene metadata columns
print("Gene metadata columns (adata.var):")
print(adata.var.columns.tolist())

In [ ]:
# View first few genes
adata.var.head()

In [ ]:
# Check expression matrix
print(f"Expression matrix shape: {adata.X.shape}")
print(f"Expression matrix type: {type(adata.X)}")
print(f"Data type: {adata.X.dtype}")

## Dataset Summary Statistics

In [ ]:
# Cell type distribution
if 'cell_type' in adata.obs.columns:
    print("Cell Type Distribution:")
    print("-" * 40)
    cell_type_counts = adata.obs['cell_type'].value_counts()
    print(cell_type_counts.head(20))
    print(f"\nTotal unique cell types: {len(cell_type_counts)}")

In [ ]:
# Tissue distribution
if 'tissue_general' in adata.obs.columns:
    print("Tissue Distribution:")
    print("-" * 40)
    print(adata.obs['tissue_general'].value_counts())

In [ ]:
# Assay distribution
if 'assay' in adata.obs.columns:
    print("Assay Distribution:")
    print("-" * 40)
    print(adata.obs['assay'].value_counts())

In [ ]:
# Disease distribution
if 'disease' in adata.obs.columns:
    print("Disease Distribution:")
    print("-" * 40)
    print(adata.obs['disease'].value_counts())

In [ ]:
# Sex distribution
if 'sex' in adata.obs.columns:
    print("Sex Distribution:")
    print("-" * 40)
    print(adata.obs['sex'].value_counts())

## Basic QC Metrics

In [ ]:
# Calculate QC metrics
print("Calculating QC metrics...")

# Identify mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('MT-')
print(f"  Found {adata.var['mt'].sum()} mitochondrial genes")

# Identify ribosomal genes
adata.var['ribo'] = adata.var_names.str.startswith(('RPS', 'RPL'))
print(f"  Found {adata.var['ribo'].sum()} ribosomal genes")

# Calculate metrics
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], inplace=True)
print("  ✓ QC metrics calculated")

In [ ]:
# QC summary statistics
print("QC Metrics Summary:")
print("=" * 50)
print(f"n_genes_by_counts:")
print(f"  Min:    {adata.obs['n_genes_by_counts'].min():,.0f}")
print(f"  Median: {adata.obs['n_genes_by_counts'].median():,.0f}")
print(f"  Max:    {adata.obs['n_genes_by_counts'].max():,.0f}")
print()
print(f"total_counts:")
print(f"  Min:    {adata.obs['total_counts'].min():,.0f}")
print(f"  Median: {adata.obs['total_counts'].median():,.0f}")
print(f"  Max:    {adata.obs['total_counts'].max():,.0f}")
print()
print(f"pct_counts_mt:")
print(f"  Min:    {adata.obs['pct_counts_mt'].min():.2f}%")
print(f"  Median: {adata.obs['pct_counts_mt'].median():.2f}%")
print(f"  Max:    {adata.obs['pct_counts_mt'].max():.2f}%")

In [ ]:
# Plot QC metrics - Violin plots
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

sc.pl.violin(adata, 'n_genes_by_counts', ax=axes[0], show=False)
axes[0].set_title('Genes per Cell')

sc.pl.violin(adata, 'total_counts', ax=axes[1], show=False)
axes[1].set_title('UMI Counts per Cell')

sc.pl.violin(adata, 'pct_counts_mt', ax=axes[2], show=False)
axes[2].set_title('% Mitochondrial')

sc.pl.violin(adata, 'pct_counts_ribo', ax=axes[3], show=False)
axes[3].set_title('% Ribosomal')

plt.tight_layout()
plt.show()

In [ ]:
# Plot QC metrics - Scatter plots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(adata.obs['total_counts'], adata.obs['n_genes_by_counts'], 
                alpha=0.1, s=1, c='steelblue')
axes[0].set_xlabel('Total UMI Counts')
axes[0].set_ylabel('Number of Genes')
axes[0].set_title('Counts vs Genes')

axes[1].scatter(adata.obs['total_counts'], adata.obs['pct_counts_mt'], 
                alpha=0.1, s=1, c='steelblue')
axes[1].set_xlabel('Total UMI Counts')
axes[1].set_ylabel('% Mitochondrial')
axes[1].set_title('Counts vs % Mito')

plt.tight_layout()
plt.show()

## Filter to Mast Cells (Optional)

In [ ]:
# Check if mast cells are present
if 'cell_type' in adata.obs.columns:
    mast_cells = adata.obs['cell_type'].str.lower().str.contains('mast')
    n_mast = mast_cells.sum()
    print(f"Mast cells in dataset: {n_mast:,}")
    
    if n_mast > 0:
        # Show mast cell types
        print("\nMast cell type labels:")
        print(adata.obs.loc[mast_cells, 'cell_type'].value_counts())

In [ ]:
# Subset to mast cells only
if 'cell_type' in adata.obs.columns:
    mast_cells = adata.obs['cell_type'].str.lower().str.contains('mast')
    
    if mast_cells.sum() > 0:
        adata_mast = adata[mast_cells, :].copy()
        print(f"Subset to mast cells: {adata_mast.n_obs:,} cells x {adata_mast.n_vars:,} genes")
    else:
        print("No mast cells found in this dataset")

## Save Processed Data

In [ ]:
# Create output directory
Path("data/processed").mkdir(parents=True, exist_ok=True)

# Save the full dataset (if you want)
# adata.write_h5ad("data/processed/dataset_with_qc.h5ad")
# print("Saved: data/processed/dataset_with_qc.h5ad")

# Save mast cell subset (if created)
if 'adata_mast' in dir() and adata_mast is not None:
    adata_mast.write_h5ad("data/processed/mast_cells.h5ad")
    print("Saved: data/processed/mast_cells.h5ad")

## Summary

In [ ]:
print("=" * 60)
print("DATASET SUMMARY")
print("=" * 60)
print(f"Total cells:      {adata.n_obs:,}")
print(f"Total genes:      {adata.n_vars:,}")
print(f"Cell types:       {adata.obs['cell_type'].nunique() if 'cell_type' in adata.obs.columns else 'N/A'}")
print(f"Tissues:          {adata.obs['tissue_general'].nunique() if 'tissue_general' in adata.obs.columns else 'N/A'}")
print(f"MT genes:         {adata.var['mt'].sum()}")
print(f"Ribo genes:       {adata.var['ribo'].sum()}")
print("=" * 60)